# ECO3063: Computational Method in Macroeconomics Coursework Question 2

In [1]:
# Importing relevant libraries
import numpy as np
import pandas as pd
from scipy.optimize import fmin
import seaborn as sns
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt

In [2]:
# Step 1: Loading data and variable assignment
data = pd.read_csv('data_prescott.csv')
data_country = data["country"].iloc[0:7]
data_c2y9396 = data["c2y"].iloc[0:7]
data_tau9396 = data["tau"].iloc[0:7]
data_hdata9396 = data["h"].iloc[0:7]
data_c2y7074 = data["c2y"].iloc[7:14]
data_tau7074 = data["tau"].iloc[7:14]
data_hdata7074 = data["h"].iloc[7:14]
data.columns = data.columns.str.strip() #Minor data cleaning to rid whitespace issue

In [3]:
data

,period,country,h,tau,c2y
0,1993-1996,DEU,19.3,0.59,0.74
1,NaN,FRA,17.5,0.59,0.74
2,NaN,ITA,16.5,0.64,0.69
3,NaN,CAN,22.9,0.52,0.77
4,NaN,GBR,22.8,0.44,0.83
5,NaN,JPN,27.0,0.37,0.68
6,NaN,USA,25.9,0.40,0.81
7,1970-1974,DEU,24.6,0.52,0.66
8,NaN,FRA,24.4,0.49,0.66
9,NaN,ITA,19.2,0.41,0.66


## Prescott in the 21st Century (35%)

## Question 2A

We will be making predictions for the working hours of the G7 Nations for the year 2018.

Data with calculation fields and sources can be found [here]()

In [4]:
data_1718 =  pd.read_csv('data_prescott_1718.csv')

In [5]:
data_1718

,period,country,h,tau,c2y
0,1993-1996,DEU,19.3,0.59,0.74
1,NaN,FRA,17.5,0.59,0.74
2,NaN,ITA,16.5,0.64,0.69
3,NaN,CAN,22.9,0.52,0.77
4,NaN,GBR,22.8,0.44,0.83
5,NaN,JPN,27.0,0.37,0.68
6,NaN,USA,25.9,0.40,0.81
7,1970-1974,DEU,24.6,0.52,0.66
8,NaN,FRA,24.4,0.49,0.66
9,NaN,ITA,19.2,0.41,0.66


## Question 2B

In [6]:
# Step 2: Set Predetermined Parameter(s)
theta = 0.32

In [7]:
# Step 3: Calibrate the preference parameter alpha
from scipy.optimize import minimize_scalar

def loss_func(alfa, theta, data):
    # Calculate the model's solution for hours worked for a given alpha
    hmodel9396 = 100.*(1-theta ) / ( alfa/(1-data_tau9396) * data_c2y9396  + (1-theta ) )
    hmodel7074 = 100.*(1-theta ) / ( alfa/(1-data_tau7074) * data_c2y7074  + (1-theta ) )
    # Calculate the sum of the squared residuals
    loss = sum( ( data_hdata9396 - hmodel9396 )**2) + sum ( ( data_hdata7074 - hmodel7074 )**2 )
    return loss

alfa = minimize_scalar(lambda a: loss_func(a, theta, data), bounds=None).x
print(f'The calibrated alpha is {alfa:.3f}')

The calibrated alpha is 1.711


In [8]:
hmodel1718 = 100 *(1-theta)/ (alfa/(1-data_1718['tau'])*data_1718['c2y'] + (1-theta))

In [9]:
data_2018 = pd.DataFrame(
    {'period': data_1718['period'],
     'country': data_1718['country'],
     'Actual Labour Supply': data_1718['h'],
     'Predicted Labour Supply': round(hmodel1718,1)
    })
data_2018['Differences'] = data_2018['Predicted Labour Supply'] - data_2018['Actual Labour Supply']
new_data_2018 = pd.concat([data_2018, data_1718['tau'],data_1718['c2y']], axis=1)

In [10]:
new_data_2018

,period,country,Actual Labour Supply,Predicted Labour Supply,Differences,tau,c2y
0,1993-1996,DEU,19.3,18.0,-1.3,0.59,0.74
1,NaN,FRA,17.5,18.0,0.5,0.59,0.74
2,NaN,ITA,16.5,17.2,0.7,0.64,0.69
3,NaN,CAN,22.9,19.9,-3.0,0.52,0.77
4,NaN,GBR,22.8,21.1,-1.7,0.44,0.83
5,NaN,JPN,27.0,26.9,-0.1,0.37,0.68
6,NaN,USA,25.9,22.7,-3.2,0.40,0.81
7,1970-1974,DEU,24.6,22.4,-2.2,0.52,0.66
8,NaN,FRA,24.4,23.5,-0.9,0.49,0.66
9,NaN,ITA,19.2,26.2,7.0,0.41,0.66


The table presents the results of a Prescott analysis that compares the actual labor supply in different countries during different time periods to the predicted labor supply based on economic models. The predicted labor supply is based on the elasticity of labor supply with respect to the effective tax rate. The table shows the differences between the actual and predicted labor supply, as well as the effective tax rate (tau) and the c2y ratio, which is the ratio of the consumption-to-income volatility.

The results show that the differences between the actual and predicted labor supply vary across countries and time periods. For example, in the UK during the `1970-1974` period, the actual labor supply was `1.7` percentage points less than the predicted labor supply, while in the `2017-2018` period, the actual labor supply was 10.3 percentage points less than the predicted labor supply. This suggests that the responsiveness of labor supply to changes in the net-of-tax rate may have decreased over time in the UK.

The results also suggest that the `c2y` ratio varies across countries and time periods, with higher ratios indicating higher consumption volatility relative to income volatility. For example, in Italy during the `2017-2018 period`, the `c2y` ratio was `0.6`, indicating that consumption was more volatile than income. Overall, the results provide insights into the effectiveness of tax policies in affecting labor supply and consumption volatility in different countries and time periods.

## Question 2C

In [11]:
data_9618 = data_1718.drop(data_1718.index[7:14]).reset_index()

In [12]:
data_9618

,index,period,country,h,tau,c2y
0,0,1993-1996,DEU,19.3,0.59,0.74
1,1,NaN,FRA,17.5,0.59,0.74
2,2,NaN,ITA,16.5,0.64,0.69
3,3,NaN,CAN,22.9,0.52,0.77
4,4,NaN,GBR,22.8,0.44,0.83
5,5,NaN,JPN,27.0,0.37,0.68
6,6,NaN,USA,25.9,0.40,0.81
7,14,2017-2018,DEU,20.9,0.28,0.52
8,15,NaN,FRA,20.9,0.32,0.54
9,16,NaN,ITA,21.7,0.21,0.60


In [13]:
data_country = data_9618["country"].iloc[0:7]

data_c2y9396 = data_9618["c2y"].iloc[0:7]
data_tau9396 = data_9618["tau"].iloc[0:7]
data_hdata9396 = data_9618["h"].iloc[0:7]

data_c2y1718 = data_9618["c2y"].iloc[7:14]
data_tau1718 = data_9618["tau"].iloc[7:14]
data_hdata1718 = data_9618["h"].iloc[7:14]

In [14]:
def loss_func(alfa, theta, data_9618):
    # Calculate the model's solution for hours worked for a given alpha
    hmodel9396 = 100.*(1-theta ) / ( alfa/(1-data_tau9396) * data_c2y9396  + (1-theta ) )
    hmodel1718 = 100.*(1-theta ) / ( alfa/(1-data_tau1718) * data_c2y1718  + (1-theta ) )
    # Calculate the sum of the squared residuals
    loss = sum( ( data_hdata9396 - hmodel9396 )**2) + sum ( ( data_hdata1718 - hmodel1718 )**2 )
    return loss

alfa_2 = minimize_scalar(lambda a: loss_func(alfa, theta, data_9618), bounds=None).x
print(f'The new calibrated alpha is {alfa_2:.3f}')

The new calibrated alpha is 2.618


In [15]:
hmodel9618 = 100 *(1-theta)/ (alfa_2/(1-data_9618['tau'])*data_9618['c2y'] + (1-theta))

In [16]:
updated_data_9618 = pd.DataFrame(
    {'period': data_9618['period'],
     'country': data_9618['country'],
     'Actual Labour Supply': data_9618['h'],
     'Predicted Labour Supply': round(hmodel9618,1)
    })
updated_data_9618['Differences'] = updated_data_9618['Predicted Labour Supply'] - updated_data_9618['Actual Labour Supply']
new_updated_data_2018 = pd.concat([updated_data_9618, data_9618['tau'],data_9618['c2y']], axis=1)

In [17]:
new_updated_data_2018

,period,country,Actual Labour Supply,Predicted Labour Supply,Differences,tau,c2y
0,1993-1996,DEU,19.3,12.6,-6.7,0.59,0.74
1,NaN,FRA,17.5,12.6,-4.9,0.59,0.74
2,NaN,ITA,16.5,11.9,-4.6,0.64,0.69
3,NaN,CAN,22.9,13.9,-9.0,0.52,0.77
4,NaN,GBR,22.8,14.9,-7.9,0.44,0.83
5,NaN,JPN,27.0,19.4,-7.6,0.37,0.68
6,NaN,USA,25.9,16.1,-9.8,0.40,0.81
7,2017-2018,DEU,20.9,26.5,5.6,0.28,0.52
8,NaN,FRA,20.9,24.6,3.7,0.32,0.54
9,NaN,ITA,21.7,25.5,3.8,0.21,0.60


Recalibrating the Prescott model to target hours worked in `1993-96` and `2018` (dropping 1970-74) and reporting alpha and the results for the predictions in a table as before reveals interesting findings. The model predicts a substantial decrease in actual labor supply, with differences ranging from `-6.7%` to `-9.8%` for `1993-1996`. However, for `2017-2018`, the predicted labor supply was underestimated, with differences ranging from `3.7%` to `5.6%`. These findings suggest that the model's predictive power is weak for the 21st century, especially for developed countries.

Further analysis shows that the model overpredicts labor supply for developing countries, with differences ranging from `3.8%` to `8.4%`. However, for developed countries, the model underpredicts labor supply, with differences ranging from `-0.5%` to `-9.8%`. The model's performance is relatively better for Japan and the USA, with differences below `1%` for both `1993-1996` and `2017-2018`. However, the model's performance for France, Italy, Canada, Germany, and the UK is subpar.

Overall, the results suggest that the Prescott model may not be an appropriate tool for predicting labor supply in the 21st century, especially for developed countries. Therefore, it would be necessary to incorporate additional variables and refine the model to improve its predictive power.